<a href="https://colab.research.google.com/github/Sasagackie/Realtime-A.I.-Object-Detector/blob/master/Realtime_AI_Component_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preface**
This note is to describe how to make a practical trained CNN model
that can deliver high detection accuracy in recognition of electronic
components. 

In [ ]:
# Below would be executed only when you mount the directory on your local PC.
from google.colab import drive
drive.mount('/gdrive')
# the project's folder
%cd /gdrive/'My Drive'/object_detection


# **Prep Before the Training**

1) Download or prepare images you want to let your model learn.

2) Label objects on your images. This will output .xml file for each image.

3) Make ***object_detection*** directory wherever you want on your local PC.

4) With *Google Colab*, mount the ***object_detection*** directory on your local PC under the /content/drive/My Drive/ directory, so that you can handle your files under the directory from your local PC. This will make the full path to the ***object_detection*** directory on *Google Colab*, like ***/content/drive/My Drive/object_detection***.

5) Make ***data*** directory under the ***object_detection*** directory.

6) Make ***annotations*** and ***images*** directories under the ***data*** directory.

7) Copy .xml files you created at **2)** under the ***object_detection/data/annotations*** directory and the image files you gathered under the ***object_detection/data/images*** directory.

8) Make ***test_labels*** and ***train_labels*** directories under the ***object_detection/data/*** directory.

9) Split 20% of total .xml files under the ***test_labesl*** directory and the rest of .xml files under the ***train_labels*** directory.
You can make it randomly by doing like below:




In [ ]:
# Command Example below is for the 2000 total .xml files. You can make an adjustment
# by changing the number for the ratio between the training and the test, 
# according to the number of total .xml files you have.
# Lists the files inside 'annotations' in a random order
# (not really random, by their hash value instead).


# Move the current dir to /content/drive/My Drive/object_detection/data/
cd /content/drive/My\ Drive/object_detection/data/

# Below is to check how many .xml files you currently have.
ls annotations/* | wc -l

# Moves the first 400/2000 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -400 | xargs -I{} mv {} test_labels/


# Moves the rest of labels '1600' labels to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/


In [ ]:
# Change the current dir.
cd /content/drive/My\ Drive/object_detection/


In [ ]:
# Installing the packages. They don't get pre-installed by default.
# Install them by running:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools

In [ ]:
# Let's select the TensorFlow version 1 rather than version 2
# because of the less dipendency issues.
%tensorflow_version 1.x

In [ ]:
# Importing libraries:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

from google.colab import files
print(tf.__version__)

In [ ]:
# This script outputs two csv files 
# by reading the .xml files under the train_labels and the test_labels dir.
# Change the current directory
!cd /content/drive/My\ Drive/object_detection/data

def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text, # Del .jpg
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

In [ ]:
# downloads the deep learning models under the object_detection directory.
# it will make a directory under the object_detection dir, called models.
!git clone --q https://github.com/tensorflow/models.git

In [ ]:
cd /content/drive/My Drive/object_detection/models/research

In [ ]:
# compils the proto buffers
!protoc object_detection/protos/*.proto --python_out=.
# exports PYTHONPATH environment var with research and slim paths
os.environ['PYTHONPATH'] += ':./:./slim/'

In [ ]:
# Install the tf_slim
!pip install --upgrade tf_slim

In [ ]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py


# **Generating the TFRecord**
Based on the csv files you have created above, the script below creates two binary files, named ***train_labels.record*** and ***test_labels.record***.
Before executing it, you make sure that:

1) The current directory is ***object_detection/models/research/***.

2) The ***DATA_BASE_PATH*** in the script is the same as the path to your ***data*** directory.

3) You have all ***classes*** listed under the ***class_text_to_int*** function in the script.


In [ ]:
#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util
%cd /content/drive/My Drive/object_detection/models

DATA_BASE_PATH = '/content/drive/My Drive/object_detection/data/'
image_dir = DATA_BASE_PATH +'images/'


def class_text_to_int(row_label):
  if row_label == 'Capacitor':
    return 1
  elif row_label == 'Diode':
    return 1
  elif row_label == 'IC':
    return 1
  elif row_label == 'Inductor':
    return 1
  elif row_label == 'LED':
    return 1
  elif row_label == 'Potentiometer':
    return 1
  elif row_label == 'Relay':
    return 1
  elif row_label == 'Resistor':
    return 1
  elif row_label == 'Transformer':
    return 1
  elif row_label == 'Transistor':
    return 1
  else:
    return 0

def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))

In [ ]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
}

# Select a model from `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'
#selected_model = 'faster_rcnn_inception_v2'

In [ ]:
#the distination folder where the model will be saved
#change this if you have a different working dir
DEST_DIR = '/content/drive/My Drive/object_detection/models/research/pretrained_model'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [ ]:
# Move to the research dir.
cd /content/drive/My\ Drive/object_detection/models/research

In [ ]:
#the logs that are created while training 
LOG_DIR = "training/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# Set the log directry
!tensorboard --logdir=/content/drive/My\ Drive/object_deteion/models/research/training/

In [ ]:
# The main training script.
# It has to be executed at /content/drive/My Drive/object_detection/models/research
# Before the execution, make sure that all the images are RGB. 
# Delete the non-RGB images from the *images* directory and 
# the corresponding record from both *test_labels.csv* and *train_labels.csv*,
# then, generate the TFRecord again.
!python3 object_detection/model_main.py \
    --pipeline_config_path=/content/drive/My\ Drive/object_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --model_dir=training/

# **Extracting Frozen Model**
For the webcam inference, you need to extract the file, named ***frozen_inference_graph.pb***, and place it on your computer.

In [ ]:
# Set the dir where the model will be saved
# output_directory should be changed to the existing dir for a new frozen model.
output_directory = './fine_tuned_model_07'

lst = os.listdir('training')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('training', last_model)

In [ ]:
# Exporting the latest model
!python /content/drive/'My Drive'/object_detection/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/drive/My\ Drive/object_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

# **Webcam Inference**

Place the three files below at the same directry on your computer:<br>



1.   ***frozen_inference_graph.pb***
2.   ***label_map.pbtxt***
3.   ***live_inference.py***

You can copy and paste the live_inference.py code below.
Then, edit the filename at "PATH_TO_FROZEN_GRAPH =" in the live_inference.py as you want.
<br>




In [ ]:
import numpy as np
import os
import tensorflow as tf
import cv2
from object_detection.utils import label_map_util 
from object_detection.utils import visualization_utils as vis_util
from PIL import ImageFont
 
# path to the frozen graph:
PATH_TO_FROZEN_GRAPH = 'frozen_inference_graph.pb'
 
# path to the label map
PATH_TO_LABEL_MAP = 'label_map.pbtxt'
 
# number of classes 
NUM_CLASSES = 90
 
cap = cv2.VideoCapture(0)
 
# reads the frozen graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
 
label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
 
# Detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:
            # Read frame from camera
            ret, image_np = cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Image Font
            ImageFont.truetype('./Arial.ttf', 30)
            # Extract image tensor
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Extract detection boxes
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Extract detection scores
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            # Extract detection classes
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            # Extract number of detections
            num_detections = detection_graph.get_tensor_by_name(
                'num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8,
                )
        # Display output
            cv2.imshow('Electronic Components Detector', cv2.resize(image_np, (1200, 800)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
 
                break

When you are ready, just execute the command on the terminal of your computer like below:


```
python live_inference.py
```
Then, you will soon see your webcam getting activated and the inference screen. Face your webcam to the circuit board and have fun!

# **Detection Verification**
The scripts below output the images with bounding boxes on components. You can feed some images you want to verify, then receive the output results with overlapped bounding boxes on components.

In [ ]:
cd /content/drive/My\ Drive/object_detection/models/research/object_detection/

In [ ]:
# Swicth the Tensorflow v.1 and v.2.
# Choose v.2 when you try the script below
%tensorflow_version 1.x

In [ ]:
# This script verifies the input images and outputs the images with the bounding boxes as the detection.
# Since this script works for all the jpg images in the directory you specify, 
# change the directory name for the input. The output images would be put in the same dir
# with the suffix, "-out.jpg" following their original file name.

# Go to cd /content/drive/My\ Drive/object_detection/models/research/object_detection/
# before do this.

# Import libraries
import os
import cv2
import numpy as np
import tensorflow.compat.v1 as tf
import sys
import glob

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import another utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Main function
def main(f):
  # Path to frozen detection graph .pb file, which contains the model that is used
  # for object detection. Change the fine name as needed.
  PATH_TO_CKPT = '/content/drive/My Drive/object_detection/TestDir/frozen_inference_graph_03.pb'

  # Path to label map file
  PATH_TO_LABELS = '/content/drive/My Drive/object_detection/TestDir/label_map.pbtxt'

  # Path to image (Change the file name needed to be verifieid.)
  PATH_TO_IMAGE = f 

  # Number of classes the object detector can identify
  NUM_CLASSES = 10 

  # Load the label map.
  label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
  categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
  category_index = label_map_util.create_category_index(categories)

  # Load the TensorFlow model into memory.
  detection_graph = tf.Graph()
  with detection_graph.as_default():
      od_graph_def = tf.compat.v1.GraphDef()
      with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
          serialized_graph = fid.read()
          od_graph_def.ParseFromString(serialized_graph)
          tf.import_graph_def(od_graph_def, name='')

      sess = tf.Session(graph=detection_graph)

  # Input tensor is the image
  image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

  # Output tensors are the detection boxes, scores, and classes
  detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

  # Each score represents level of confidence for each of the objects.
  # The score is shown on the result image, together with the class label.
  detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
  detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

  # Number of objects detected
  num_detections = detection_graph.get_tensor_by_name('num_detections:0')

  # Load image using OpenCV and
  image = cv2.imread(PATH_TO_IMAGE)
  image_expanded = np.expand_dims(image, axis=0)

  # Perform the actual detection by running the model with the image as input
  (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections], feed_dict={image_tensor: image_expanded})

  # Draw the results of the detection
  vis_util.visualize_boxes_and_labels_on_image_array(
          image,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=0.3)
  # All the results have been drawn on image. Now display the image.
  # cv2.imshow('Object detector', cv2.resize(image, (int(2592/2),int(1944/2))))

  # # Press any key to close the image
  # cv2.waitKey(0)

  # # Clean up
  # cv2.destroyAllWindows()
  # Change the file name needed to be output.
  cv2.imwrite(f + "-out.jpg", image)

# Execute main()
files = glob.glob("/content/drive/My Drive/object_detection/TestDir/TestImages02/*.jpg")
for jpg in files:
  main(jpg)